In [1]:
import pandas as pd
import numpy as np
import random
import sys
sys.path.append('../..')
from modules.many_features import utils, constants
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

C:\Users\User\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning:

`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:167: DeprecationWarning:

`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `

In [2]:
feat_thresh_arr = [{'name':'hemoglobin', 'thresh':[12], 'std':2},
                   {'name':'ferritin', 'thresh':[30, 100], 'std':2},
                   {'name':'ret_count', 'thresh':[2], 'std':0.2},
                   {'name':'segmented_neutrophils', 'thresh':[0], 'std':2},
                   {'name':'tibc', 'thresh':[450], 'std':50},
                   {'name':'mcv', 'thresh':[80,100], 'std':2}]

anemias_features_dict = {'Vitamin B12/Folate deficiency anemia': ['mcv', 'segmented_neutrophils'], 
                         'Unspecified anemia': ['mcv', 'segmented_neutrophils'], 
                         'Anemia of chronic disease': ['mcv', 'ferritin', 'tibc'], 
                         'Iron deficiency anemia': ['mcv', 'ferritin', 'tibc'], 
                         'Hemolytic anemia': ['mcv', 'ret_count'], 
                         'Aplastic anemia': ['mcv', 'ret_count']}

#### The data

In [3]:
all_df= pd.read_csv('../../final/data/data_correlated_feats.csv')
#all_df = pd.read_csv('../../final/data/train_set_missing_2.csv')
all_df = all_df.fillna(-1)
all_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,11.875670,161.104070,3.981397,-1.000000,358.191297,77.370085,-1.000000,4.604753,0,0.508604,86.888802,-1.000000,-1.000000,-1.000000,-1.000000,35.627010,-1.000000,Anemia of chronic disease
1,9.296779,457.149834,1.874231,5.544915,-1.000000,78.717345,-1.000000,3.543099,0,0.784442,37.659905,-1.000000,-1.000000,-1.000000,-1.000000,27.890337,-1.000000,Anemia of chronic disease
2,9.780377,483.216550,4.041312,5.515052,304.155693,100.750651,118.428782,2.912252,1,1.129411,140.917968,47.714319,62.722981,20.039071,46.973234,29.341132,38.936895,Vitamin B12/Folate deficiency anemia
3,9.131688,57.578776,5.778568,1.976049,207.684776,96.855220,-1.000000,2.828455,0,1.569724,26.393979,-1.000000,-1.000000,-1.000000,-1.000000,27.395064,-1.000000,Hemolytic anemia
4,11.395680,-1.000000,-1.000000,1.320481,318.250045,102.206351,-1.000000,3.344904,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,34.187041,-1.000000,Vitamin B12/Folate deficiency anemia


In [4]:
len(all_df)

70000

In [5]:
class_dict = constants.CLASS_DICT
#print(all_df.label.unique())
all_df['label'] = all_df['label'].replace(class_dict)
X = all_df.iloc[:, 0:-1]
y = all_df.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
X_train_df = pd.concat([X_train, y_train], axis=1)
X_test_df = pd.concat([X_test, y_test], axis=1)
X_test_df = X_test_df.reset_index(drop=True)
X_train_df = X_train_df.reset_index(drop=True)
X_test_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,7.116363,-1.000000,3.781573,2.738413,-1.000000,95.904198,68.457895,2.226085,0,1.892912,39.808550,110.329197,64.404350,21.654404,73.787009,21.349089,-1.000000,5
1,8.125320,92.230003,4.231419,1.188039,143.365567,104.057204,204.747831,2.342554,0,0.652614,13.478089,-1.000000,32.705481,-1.000000,43.520272,24.375961,142.815207,1
2,11.309450,38.324563,-1.000000,-1.000000,455.077909,76.402602,-1.000000,4.440732,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,33.928350,-1.000000,4
3,13.763858,253.513394,2.262606,0.551444,453.772884,82.781943,90.101466,4.987993,0,0.853521,104.005514,34.639227,0.963866,22.083012,88.891838,41.291574,19.856071,0
4,11.464002,-1.000000,-1.000000,-1.000000,320.964653,104.287127,-1.000000,3.297819,0,1.163516,121.616315,105.895897,-1.000000,9.337462,-1.000000,34.392007,-1.000000,7


In [6]:
X_train_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,9.007012,-1.000000,-1.000000,3.519565,440.499323,103.442762,59.017997,2.612173,1,0.650757,114.794964,112.308159,25.612786,5.969710,116.026042,27.021037,13.397977,1
1,8.760976,-1.000000,0.491469,-1.000000,259.895852,103.885481,-1.000000,2.529991,0,0.728641,74.824352,-1.000000,-1.000000,-1.000000,-1.000000,26.282929,-1.000000,7
2,7.490324,70.812609,-1.000000,1.495604,482.109919,79.543391,-1.000000,2.824995,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,22.470972,-1.000000,4
3,8.113370,499.313483,3.507612,0.000000,195.351883,100.545858,203.895756,2.420797,1,0.860863,9.120947,41.131511,1.843890,18.845473,106.199806,24.340111,104.373581,2
4,13.935301,349.569415,5.190725,6.894195,489.595939,102.234294,150.085853,4.089225,1,0.216907,20.344863,92.547095,19.815123,29.543875,98.388710,41.805903,30.655044,0


In [7]:
# X_train_df.to_csv('../../final/data/train_set_basic.csv', index=False)

In [8]:
# X_test_df.to_csv('../../final/data/test_set_constant.csv', index=False)

In [9]:
def get_dict(dict_arr, feat_name):
    for dictionary in dict_arr:
        if dictionary['name']==feat_name:
            return dictionary

In [10]:
def feat_noisiness(df, feat_name, frac, mean, std):
    df[feat_name] = df[feat_name].sample(frac=1-frac)
    if feat_name ==  'segmented_neutrophils': #how was this a good idea
        df[feat_name] = df[feat_name].fillna(-1)
    else:
        nan_index = df[df[feat_name].isna()].index
        fill_values = np.random.normal(mean, std, size=len(nan_index))
        df[feat_name] = df[feat_name].fillna(pd.Series(fill_values, index=nan_index))
    return df[feat_name]

In [11]:
total_frac = 0.6

In [12]:
noisy_anemia_dfs_list = []
for label in X_train_df.label.unique():
    anemia = constants.ACTION_SPACE[label]
    print(f'{label} - {constants.ACTION_SPACE[label]}')
    if anemia not in ['No anemia', 'Inconclusive diagnosis']:
        anemia_df = X_train_df[X_train_df.label==label]
        feature_num = len(anemias_features_dict[anemia])
        frac = total_frac/feature_num
        for feat in anemias_features_dict[anemia]:
            feat_info = get_dict(feat_thresh_arr, feat)
            threshes, std = feat_info['thresh'], feat_info['std']
            print(f'feat:{feat}, threshes:{threshes}, std:{std}')
            for thresh in threshes:
                anemia_df[feat] = feat_noisiness(anemia_df, feat, frac/len(threshes), thresh, std)
        noisy_anemia_dfs_list.append(anemia_df)

1 - Vitamin B12/Folate deficiency anemia
feat:mcv, threshes:[80, 100], std:2
feat:segmented_neutrophils, threshes:[0], std:2
7 - Inconclusive diagnosis
4 - Iron deficiency anemia
feat:mcv, threshes:[80, 100], std:2
feat:ferritin, threshes:[30, 100], std:2
feat:tibc, threshes:[450], std:50
2 - Unspecified anemia
feat:mcv, threshes:[80, 100], std:2
feat:segmented_neutrophils, threshes:[0], std:2
0 - No anemia
6 - Aplastic anemia
feat:mcv, threshes:[80, 100], std:2
feat:ret_count, threshes:[2], std:0.2
5 - Hemolytic anemia
feat:mcv, threshes:[80, 100], std:2
feat:ret_count, threshes:[2], std:0.2
3 - Anemia of chronic disease
feat:mcv, threshes:[80, 100], std:2
feat:ferritin, threshes:[30, 100], std:2
feat:tibc, threshes:[450], std:50


In [13]:
noisy_anemia_dfs_list[0].describe()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
count,7203.000000,7203.000000,7203.000000,7203.000000,7203.000000,7203.000000,7203.000000,7203.000000,7203.000000,7203.000000,7203.000000,7203.000000,7203.000000,7203.000000,7203.000000,7203.000000,7203.000000,7203.0
mean,9.510462,114.590630,0.450322,2.172466,222.439107,99.218939,60.740629,2.783978,0.570596,0.515078,53.875010,27.921594,17.196119,4.825462,39.743601,28.531386,17.387749,1.0
std,2.012629,159.495280,2.190672,2.657682,170.697292,7.642506,81.393837,0.590872,0.495026,1.039171,50.050921,42.471626,25.528546,9.353014,49.167522,6.037887,34.929946,0.0
min,6.001058,-1.000000,-1.000000,-1.000000,-1.000000,73.646592,-1.000000,1.717497,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,18.003175,-1.000000,1.0
25%,7.765217,-1.000000,-1.000000,-1.000000,-1.000000,100.252221,-1.000000,2.269675,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,23.295651,-1.000000,1.0
50%,9.513341,-1.000000,-1.000000,2.041008,227.841189,101.695161,-1.000000,2.787632,1.000000,0.755746,46.352713,-1.000000,-1.000000,-1.000000,-1.000000,28.540022,-1.000000,1.0
75%,11.278146,232.291861,1.846724,4.510439,369.421111,103.351304,125.163190,3.297849,1.000000,1.369483,97.749980,60.103302,34.952750,9.295972,84.215031,33.834438,25.853717,1.0
max,12.999126,499.887714,5.999175,6.999834,519.904998,105.525369,249.884526,3.898967,1.000000,1.999901,149.989051,129.998850,79.988473,29.993930,139.998628,38.997377,239.491690,1.0


In [14]:
no_df = X_train_df[X_train_df.label==constants.CLASS_DICT['No anemia']]
inconc_df = X_train_df[X_train_df.label==constants.CLASS_DICT['Inconclusive diagnosis']]
len(no_df), len(inconc_df)

(8000, 5377)

In [15]:
noisy_anemia_dfs_list.append(no_df)
noisy_anemia_dfs_list.append(inconc_df)
len(noisy_anemia_dfs_list)

8

In [16]:
combined_df = pd.concat(noisy_anemia_dfs_list, axis=0)
combined_df = combined_df.sample(frac=1).reset_index(drop=True)
combined_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,13.770995,215.521708,4.809373,2.295072,362.932543,78.283018,-1.000000,5.277388,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,41.312985,-1.000000,0
1,12.732583,65.065199,0.260009,1.916606,462.688298,75.112867,78.127529,5.085380,1,1.894743,59.073552,129.390890,14.556256,29.659077,71.704527,38.197750,16.885564,4
2,10.393676,-1.000000,3.827465,0.000000,-1.000000,100.334310,66.644312,3.107713,1,1.894468,102.808633,63.310143,4.534010,18.751363,65.040251,31.181027,-1.000000,2
3,11.477997,-1.000000,0.652729,0.422517,-1.000000,87.038370,-1.000000,3.956185,1,1.976823,132.499051,-1.000000,-1.000000,-1.000000,-1.000000,34.433992,-1.000000,6
4,15.256418,-1.000000,3.076288,2.469028,135.052614,-1.000000,214.091425,-1.000000,0,1.247224,6.391319,69.744325,76.960520,15.376436,132.024562,45.769255,158.524458,0


#### The correlated features

In [17]:
def get_rbc(row):
    if row['mcv'] == -1:
        return -1
    else:
        return (30*row['hemoglobin'])/row['mcv']

def get_tsat(row):
    if (row['serum_iron']==-1) | (row['tibc']==-1):
        return -1
    else:
        return (row['serum_iron']/row['tibc'])*100

In [18]:
combined_df['rbc'] = combined_df.apply(lambda row: get_rbc(row), axis=1)
combined_df['tsat'] = combined_df.apply(lambda row: get_tsat(row), axis=1)
combined_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,13.770995,215.521708,4.809373,2.295072,362.932543,78.283018,-1.000000,5.277388,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,41.312985,-1.000000,0
1,12.732583,65.065199,0.260009,1.916606,462.688298,75.112867,78.127529,5.085380,1,1.894743,59.073552,129.390890,14.556256,29.659077,71.704527,38.197750,16.885564,4
2,10.393676,-1.000000,3.827465,0.000000,-1.000000,100.334310,66.644312,3.107713,1,1.894468,102.808633,63.310143,4.534010,18.751363,65.040251,31.181027,-1.000000,2
3,11.477997,-1.000000,0.652729,0.422517,-1.000000,87.038370,-1.000000,3.956185,1,1.976823,132.499051,-1.000000,-1.000000,-1.000000,-1.000000,34.433992,-1.000000,6
4,15.256418,-1.000000,3.076288,2.469028,135.052614,-1.000000,214.091425,-1.000000,0,1.247224,6.391319,69.744325,76.960520,15.376436,132.024562,45.769255,158.524458,0


#### Randomly modifying to no anemia

In [19]:
anemic_df = combined_df[combined_df.label != constants.CLASS_DICT['No anemia']]
non_anemic_df = combined_df[combined_df.label == constants.CLASS_DICT['No anemia']]
len(anemic_df), len(non_anemic_df)

(48000, 8000)

In [20]:
anemic_df.label.value_counts()

6    7223
5    7220
1    7203
2    7171
3    7087
4    6719
7    5377
Name: label, dtype: int64

In [21]:
anemic_df['label'] = anemic_df['label'].sample(frac=1-0.1)
anemic_df['label'] = anemic_df['label'].fillna(constants.CLASS_DICT['No anemia'])
anemic_df.label.value_counts()

6.0    6528
5.0    6468
1.0    6465
2.0    6464
3.0    6397
4.0    6053
7.0    4825
0.0    4800
Name: label, dtype: int64

#### Finalizing 

In [22]:
final_df = pd.concat([non_anemic_df, anemic_df], axis=0)
final_df = final_df.sample(frac=1).reset_index(drop=True)
final_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,9.401651,-1.000000,5.840045,0.000000,116.415615,103.966653,-1.000000,2.712885,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,28.204953,-1.000000,2.0
1,7.029873,102.717562,2.079564,-1.000000,-1.000000,79.373697,212.817785,2.657004,1,0.675576,104.688780,50.452242,59.237694,8.549773,49.832026,21.089620,-1.000000,3.0
2,15.900836,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,58.363850,-1.000000,1,0.999932,90.916733,60.845387,77.678965,1.513323,139.638756,47.702509,-1.000000,0.0
3,12.710083,437.896009,2.773303,-1.000000,392.155888,103.338649,91.089902,3.689834,1,1.431816,31.576192,124.321572,79.407308,24.911028,84.817826,38.130249,23.227983,7.0
4,9.256332,267.393719,2.347146,5.758205,258.911569,76.491274,189.160039,3.630348,1,1.259486,19.493158,-1.000000,55.681222,-1.000000,96.976233,27.768995,73.059709,3.0


In [23]:
final_df.label.value_counts()

0.0    12800
6.0     6528
5.0     6468
1.0     6465
2.0     6464
3.0     6397
4.0     6053
7.0     4825
Name: label, dtype: int64

In [24]:
final_df.to_csv('../../final/data/train_set_noisy_6.csv', index=False)

In [25]:
utils.get_dt_performance(final_df, 'numeric')

(0.6408928571428572,
 0.6394570966049525,
 0.792359773581911,
 datetime.timedelta(microseconds=2001))

In [26]:
len(final_df)

56000